---
#### 작업형 1-1. 결측치가 있는 행을 제거한 후 학생이 가장 많이 수강한 과목(id_assessment)을 찾고, 해당 과목점수를 표준화(StandardScaler)한 후, 가장 큰 표준화된 값을 구하라 (반올림하여 소수 3째자리)
---

In [2]:
import numpy as np
import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/student_assessment.csv")
#display(df.shape, df.head(2))

# 결측치 제거
df_na = df.dropna()
# 가장많이 수강한 과목
df_na['id_assessment'].idxmax() # 12
df_new = df_na[df_na['id_assessment'] == 12]
# 표준화
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_new['score'] = scaler.fit_transform(df_new[['score']])
print('정답 = ', round(df_new['score'].max(), 3))

정답 =  2.183


/tmp/ipython-input-2536331741.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['score'] = scaler.fit_transform(df_new[['score']])


---
#### 작업형 1-2. DE1~DE77 컬럼 중 주가지수의 종가 close와 가장 상관관계가 높은 변수를 찾아 해당변수의 평균값을 구하라 (반올림, 소수 4자리)
---

In [3]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/stock_market.csv")
display(df.shape, df.head())

corr = df.corr()['close'].drop(['close', 'open', 'volume'])
corr.abs().idxmax() ## 상관관계가 가장 높은 변수 = DE14

print('정답 = ', round(df['DE14'].mean(), 4))

(1000, 80)

,DE1,DE2,DE3,DE4,DE5,DE6,DE7,DE8,DE9,DE10,...,DE71,DE72,DE73,DE74,DE75,DE76,DE77,open,close,volume
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599,...,0.729091,0.128983,1.139401,-1.234826,0.402342,-0.684810,-0.870797,129.549102,126.502056,981466.0
1,-0.578850,-0.311553,0.056165,-1.165150,0.900826,0.465662,-1.536244,1.488252,1.895889,1.178780,...,1.117016,-1.315907,-0.461585,-0.068242,1.713343,-0.744755,-0.826439,126.502056,128.041811,473416.0
2,-0.098453,-0.663478,1.126636,-1.079932,-1.147469,-0.437820,-0.498032,1.929532,0.949421,0.087551,...,-1.540797,0.063262,0.156507,0.232181,-0.597316,-0.237922,-1.424061,128.041811,128.957651,46597.0
3,-0.493320,-0.542861,0.416050,-1.156182,0.781198,1.494485,-2.069985,0.426259,0.676908,-0.637437,...,1.658131,-0.118164,-0.680178,0.666383,-0.460720,-1.334258,-1.346718,128.957651,122.965246,455747.0
4,0.693773,-0.159573,-0.133702,1.077744,-1.126826,-0.730678,-0.384880,0.094352,-0.042171,-0.286887,...,-0.206904,0.880179,-1.698106,0.387280,-2.255564,-1.022507,0.038631,122.965246,114.922790,242198.0


정답 =  -0.0004


---
#### 작업형 1-3. IQR을 이용해 이산화탄소(CO2)의 이상치 수를 찾아라
- IQR = Q3-Q1 / 이상치 = Q1-1.5*IQR ~ Q3+1.5*IQR
---

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/air_quality.csv")
display(df.shape, df.head())

q1 = df['CO2'].quantile(0.25)
q3 = df['CO2'].quantile(0.75)
iqr = q3 - q1
lower = q1 - (1.5 * iqr)
upper = q3 + (1.5 * iqr)

df = df[
    (df['CO2'] < lower) | (df['CO2'] > upper)
]
print('정답 = ', len(df))

(10000, 4)

,Date,CO,CO2,O2
0,2024-01-01 00:00:00,0.549671,383.037632,21.248286
1,2024-01-01 01:00:00,0.486174,392.362513,21.183324
2,2024-01-01 02:00:00,0.564769,385.065473,19.963480
3,2024-01-01 03:00:00,0.652303,402.760451,21.479584
4,2024-01-01 04:00:00,0.476585,429.929463,19.409917


정답 =  304


---
#### 작업형 2-1. W-mart 매장의 판매데이터를 기반으로 판매액을 예측하라
- 타겟: total
- 평가: RMSE
- 제출: result.csv (pred)  
---

In [5]:
train = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_train.csv")
test = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/mart_test.csv")
display(train.shape, test.shape, train.head(2), test.head(2))

############# EDA ##########################
# 결측치 없음
# 범주형 변수 다수, 범주형 변수 카테고리 동일
############################################

# 1. 타겟분리
y = train['total']
train = train.drop('total', axis=1)

# 2. 원-핫 인코딩 (수치형변수는 rating 하나이고, 분포가 비교적 균질해서 스케일링 스킵)
train = pd.get_dummies(train)
test = pd.get_dummies(test)

# 3. 검증데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.2, random_state=42)

# 4. model, eval
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error
model = RandomForestRegressor(random_state=120)
model.fit(X_train, y_train)

model_pred = model.predict(X_valid)
score = root_mean_squared_error(y_valid, model_pred)
print('RMSE = ', score)

(700, 10)

(300, 9)

,branch,city,customer_type,gender,product_line,total,payment_method,rating,time_of_day,day_name
0,A,Yangon,Member,Female,Health and beauty,823457.25,Ewallet,9.1,afternoon,Saturday
1,C,Naypyitaw,Normal,Female,Electronic accessories,120330.00,Cash,9.6,morning,Friday


,branch,city,customer_type,gender,product_line,payment_method,rating,time_of_day,day_name
0,C,Naypyitaw,Normal,Female,Fashion accessories,Ewallet,9.6,afternoon,Thursday
1,B,Mandalay,Normal,Male,Food and beverages,Credit card,4.3,evening,Wednesday


RMSE =  408119.1236985218


In [6]:
## model2 : LinearRegerssion
from sklearn.linear_model import LinearRegression
model2 = LinearRegression()
model2.fit(X_train, y_train)

model2_pred = model2.predict(X_valid)
lr_score = root_mean_squared_error(y_valid, model2_pred)
print('lr RMSE = ', lr_score)

lr RMSE =  399395.51835163427


In [7]:
pred = model2.predict(test)
result = pd.DataFrame({
    'pred':pred
})
result.to_csv('result.csv', index=False)

---
#### 작업형 3-1. 조개의 정보를 나타낸 데이터임. 300개 중 앞에서 부터 210개는 train, 90개는 test로 분리하라. 다중 로지스틱 회귀분석을 통해 성별(gender)를 예측하되 페널치는 부과하지 않는다.
- 종속변수: gender (0:암컷, 1:수컷)
1. 독립변수 weight로 로지스틱 모델을 만들고, odds비를 구하라
2. age, length, diameter,height, weight에 대해 로지스틱 모델을 만들고 잔차이탈도를 구하라
3. 독립변수 weight만 사용하나 모델에서 test의 gender를 예측하고, Error Rate를 구하라
---

In [21]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/clam.csv")
display(df.shape, df.head())

train = df[:210]
test = df[210:]
test_y = test['gender']
test = test.drop('gender', axis=1)

from statsmodels.formula.api import ols, logit
model = logit('gender ~ weight', data=train).fit()
display(model.summary())

odds_ratio = np.exp(0.0047)
print('1. odds ratio = ', odds_ratio)

(300, 6)

,age,length,diameter,height,weight,gender
0,6,0.474627,0.211352,0.178189,78.971766,1
1,1,0.465847,0.339388,0.170522,98.781960,1
2,4,0.122807,0.238691,0.106924,88.792625,0
3,4,0.204579,0.360543,0.034261,1.028847,0
4,8,0.243458,0.358037,0.128080,6.503367,0


Optimization terminated successfully.
         Current function value: 0.690045
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      208
Method:                           MLE   Df Model:                            1
Date:                Mon, 15 Sep 2025   Pseudo R-squ.:                0.003431
Time:                        14:59:27   Log-Likelihood:                -144.91
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.3178
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3140      0.276     -1.137      0.256      -0.855       0.227
weight         0.0047      0.005      0.997      0.319      -0.005       0.014
==============================================================================
"""

1. odds ratio =  1.0047110623241844


In [22]:
model2 = logit('gender ~ age + length + diameter + height + weight', data=train).fit()
display(model2.summary())
# 잔차이탈도 = Log-Likelihood * -2
print('2.잔차이탈도 = ', -143.47 * -2)

Optimization terminated successfully.
         Current function value: 0.683173
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                 gender   No. Observations:                  210
Model:                          Logit   Df Residuals:                      204
Method:                           MLE   Df Model:                            5
Date:                Mon, 15 Sep 2025   Pseudo R-squ.:                 0.01336
Time:                        14:59:30   Log-Likelihood:                -143.47
converged:                       True   LL-Null:                       -145.41
Covariance Type:            nonrobust   LLR p-value:                    0.5662
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6701      0.651      1.030      0.303      -0.605       1.946
age           -0.0482      0.055     -0.881      0.378      -0.156       0.059
length        -0.7832      0.998     -0.785      0.433      -2.739       1.173
diameter      -1.1573      1.389     -0.833      0.405      -3.880       1.565
height        -2.2977      2.640     -0.870      0.384      -7.472       2.877
weight         0.0054      0.005      1.098      0.272      -0.004       0.015
==============================================================================
"""

2.잔차이탈도 =  286.94


In [23]:
from sklearn.metrics import accuracy_score
pred = model.predict(test)
pred = (pred > 0.5).astype(int)
score = accuracy_score(test_y, pred)
# Error rate = 1-accuracy
print('3.Error rate = ', 1-score)

3.Error rate =  0.47777777777777775


---
#### 작업형 3-2. 선형회귀
1. ERP와 가장 상관관계가 높은 값은? (반올림, 소수3째자리)
2. CPU컬럼이 100미만인 것을 찾아 ERP를 종속변수, 나머지를 독립변수로 선형회귀모델을 만들고 결정계수를 구하라 (반올림, 소수3자리)
3. 위 모델에서 독립변수 중 p값이 가장 높은 것은? (반올림, 소수3자리)
---

In [9]:
df = pd.read_csv("https://raw.githubusercontent.com/lovedlim/bigdata_analyst_cert/main/part4/ch7/system_cpu.csv")
display(df.shape, df.head())

(115, 5)

,ERP,Feature1,Feature2,Feature3,CPU
0,30.6,235.1,44.5,44.0,112.3
1,40.3,36.6,46.4,36.1,58.6
2,57.7,52.2,66.5,2.0,55.3
3,128.3,196.2,59.8,57.4,103.2
4,80.3,75.2,59.6,58.2,104.1


In [10]:
corr = df.corr()['ERP'].drop('ERP') #Feature3
print('1.ERP와 가장 상관관계가 높은 값 = ', round(corr.abs().max(), 3) )

1.ERP와 가장 상관관계가 높은 값 =  0.882


In [11]:
df_new = df[df['CPU'] < 100 ]
from statsmodels.formula.api import ols
model = ols('ERP ~ Feature1 + Feature2 + Feature3 + CPU',data=df_new).fit()
display(model.summary())

print('2.결정계수 = ', 0.755)
print('3.독립변수 중 p값이 가장 높은 것 =', 0.684)

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    ERP   R-squared:                       0.755
Model:                            OLS   Adj. R-squared:                  0.736
Method:                 Least Squares   F-statistic:                     39.30
Date:                Mon, 15 Sep 2025   Prob (F-statistic):           5.36e-15
Time:                        14:52:45   Log-Likelihood:                -260.40
No. Observations:                  56   AIC:                             530.8
Df Residuals:                      51   BIC:                             540.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     51.4133     19.112      2.690      0.010      13.045      89.782
Feature1      -0.0242      0.059     -0.409      0.684      -0.143       0.094
Feature2       0.0602      0.106      0.569      0.572      -0.152       0.273
Feature3       1.4126      0.113     12.458      0.000       1.185       1.640
CPU           -0.4651      0.234     -1.985      0.053      -0.936       0.005
==============================================================================
Omnibus:                        3.758   Durbin-Watson:                   1.762
Prob(Omnibus):                  0.153   Jarque-Bera (JB):                2.757
Skew:                           0.436   Prob(JB):                        0.252
Kurtosis:                       3.648   Cond. No.                         780.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

2.결정계수 =  0.755
3.독립변수 중 p값이 가장 높은 것 = 0.684
